In [64]:
from DeepMID import *
from augment import *
from readBruker import *

# load the model

In [4]:
model_save_path = 'model/'
model_name = 'test_nmr'
model = load_DeepMID(model_save_path + model_name)
print("Load the model successfully")

Load the model successfully


In [29]:
plant_flavors = read_bruker_hs_base('data/plant_flavors', False, True, False)
known_formulated_flavors = read_bruker_hs_base('data/known_formulated_flavors', False, True, False)

Read Bruker H-NMR files: 100%|██████████| 16/16 [00:03<00:00,  4.13it/s]


In [61]:
def predict_test():
    pickle_file_test = open('data/data_augment_test.pkl', 'rb')
    aug_test = pickle.load(pickle_file_test)
    ev = evaluate_DeepMID(model, [aug_test['R'], aug_test['S']], aug_test['y'])
    yp_test = predict_DeepMID(model, [aug_test['R'], aug_test['S']])
    yp_test_list = [1 if yp_test[i, 0] >= 0.5 else 0 for i in range(yp_test.shape[0])]
    yp_test = np.array(yp_test_list).reshape(yp_test.shape)
    cnf_matrix = confusion_matrix(aug_test['y'], yp_test)
    print('test_set_cnf_matrix', ' \n', cnf_matrix)
    return cnf_matrix


def predict_known():
    for i in range(16):
        query = known_formulated_flavors[i]

        p = query['ppm'].shape[0]
        n = len(plant_flavors)
        R = np.zeros((n, p), dtype=np.float32)
        Q = np.zeros((n, p), dtype=np.float32)
        for i in range(n):
            R[i,] = plant_flavors[i]['fid']
            Q[i,] = query['fid']
        yp = predict_DeepMID(model, [R, Q])

        plant_flavors_df = pd.read_csv('data/plant_flavors.csv', encoding='gb2312')
        result_df = pd.DataFrame(columns=['Name', 'Probability'])

        for t in range(n):
            result_df.loc[len(result_df)] = [plant_flavors[t]['name'], yp[t][0]]

        result = pd.merge(plant_flavors_df['Name'], result_df, on=['Name'])
        result1 = result.sort_values(by=['Probability'], ascending=False)
        print(query['name'], ' \n', result1.loc[:, 'Name':'Probability'])

# train model

In [27]:
pickle_file = open('data/data_augment_train.pkl', 'rb')
aug = pickle.load(pickle_file)

pickle_file_valid = open('data/data_augment_valid.pkl', 'rb')
aug_valid = pickle.load(pickle_file_valid)

model = DeepMID([aug['R'].shape, aug['S'].shape], 8)
# DeepMID(xshapes, num_conv_layers-1)
train_DeepMID(model, [aug['R'], aug['S']], aug['y'], 64, 100, [aug_valid['R'], aug_valid['S']], aug_valid['y'])
# train_DeepMID(model, Xs, y, batch, epochs, Xs_valid, y_valid):
save_DeepMID(model, model_save_path + model_name)

model = load_DeepMID(model_save_path + model_name)
plot_loss_accuracy(model)

# test set
predict_test()

# known formulated flavors
predict_known()

# predict result

In [62]:
model = load_DeepMID(model_save_path + model_name)

# test set
predict_test()

# known formulated flavors
predict_known()

313/313 [==============================] - 11s 30ms/step - loss: 0.0140 - accuracy: 0.9958
test_set_cnf_matrix  
 [[4984   16]
 [  26 4974]]
B1  
                                  Name   Probability
4          MZTQW_MeOD_MeOD_600_128_wt  1.000000e+00
5         PJHTQW_MeOD_MeOD_600_128_wt  1.000000e+00
7      WHGTQW_HF_MeOD_MeOD_600_128_wt  1.000000e+00
11              GPTQW_HF_600_128_wt_2  1.000000e+00
2   LMCHJTQWX_HF_MeOD_MeOD_600_128_wt  9.907722e-01
1       JJTQW_HF_MeOD_MeOD_600_128_wt  9.611291e-01
3       MXTQW_HF_MeOD_MeOD_600_128_wt  1.257524e-01
0          JDTQW_MeOD_MeOD_600_128_wt  4.543517e-06
8         XCGTQW_MeOD_MeOD_600_128_wt  6.665644e-27
9       YCMLDFYW_MeOD_MeOD_600_128_wt  1.213578e-37
6         PTGTQW_MeOD_MeOD_600_128_wt  0.000000e+00
10   LMCHJTQW_GL_MeOD_MeOD_600_128_wt  0.000000e+00
12    YNYCTQW_HF_MeOD_MeOD_600_128_WT  0.000000e+00
B2  
                                  Name   Probability
5         PJHTQW_MeOD_MeOD_600_128_wt  1.000000e+00
7      WHGTQW_H

# data augment

In [ ]:
from readBruker import read_bruker_hs_base

plant_flavors = read_bruker_hs_base('data/plant_flavors', False, True, False)

aug_train = data_augmentation(plant_flavors, 40000, 5)
# data_augmentation(spectra, amount, max_plant_flavors)
load_train_data(aug_train)

pickle_file = open('data_augment_train.pkl', 'rb')
aug_train = pickle.load(pickle_file)

aug_valid = data_augmentation(plant_flavors, 5000, 5)
load_valid_data(aug_valid)

pickle_file = open('data_augment_valid.pkl', 'rb')
aug_valid = pickle.load(pickle_file)

aug_test = data_augmentation(plant_flavors, 5000, 5)
load_valid_data(aug_test)

pickle_file = open('data_augment_test.pkl', 'rb')
aug_test = pickle.load(pickle_file)